In [1]:
from patternq.dataset import clinical_observations_for_patients
%load_ext autoreload
%autoreload 2
import pandas as pd
import patternq.query as pqq
import patternq.dataset as pqd
import patternq.reference as pqr

/Users/benjaminkamphaus/Library/Caches/pypoetry/virtualenvs/patternq-bsnIQB4Y-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
subjects = pqd.all_subjects('tcga-brca', db_name='tcga-brca')
subjects

ConnectionError: HTTPConnectionPool(host='data-commons-query-dev.us-east-1.elasticbeanstalk.com', port=80): Max retries exceeded with url: /query/tcga-brca (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x11842b3a0>: Failed to resolve 'data-commons-query-dev.us-east-1.elasticbeanstalk.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [3]:
by_hla = subjects[['subject-id', 'subject-HLA-A-type', 'subject-HLA-B-type', 'subject-HLA-C-type']].explode(column='subject-HLA-A-type')
by_hla = by_hla.explode(column='subject-HLA-B-type')
by_hla = by_hla.explode(column='subject-HLA-C-type')
by_hla

,subject-id,subject-HLA-A-type,subject-HLA-B-type,subject-HLA-C-type
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_40_01_02,hla_c_03_04_01_01
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_40_01_02,hla_c_07_04_01
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_44_02_01_03,hla_c_03_04_01_01
0,TCGA-AR-A1AR,hla_a_68_01_02,hla_b_44_02_01_03,hla_c_07_04_01
1,TCGA-BH-A1EO,hla_a_25_01_01,hla_b_18_01_01_02,hla_c_12_03_01_01
...,...,...,...,...
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_27_05_02,hla_c_03_04_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_27_05_02,hla_c_06_02_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_40_01_02,hla_c_03_04_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_40_01_02,hla_c_06_02_01_01


In [5]:
by_hla['subject-HLA-B-type'].value_counts()

subject-HLA-B-type
hla_b_07_02_01       841
hla_b_08_01_01       508
hla_b_15_01_01_01    423
hla_b_44_02_01_01    393
hla_b_35_01_01_02    332
                    ... 
hla_b_51_07_02         2
hla_b_39_01_03         1
hla_b_44_02_15         1
hla_b_14_03            1
hla_b_07_02_06         1
Name: count, Length: 102, dtype: int64

In [10]:
b27_matches = by_hla['subject-HLA-B-type'].astype('string').str.contains("b_27")
b27_matches.value_counts()
b27_sub = by_hla[b27_matches]
b27_sub

,subject-id,subject-HLA-A-type,subject-HLA-B-type,subject-HLA-C-type
15,TCGA-E2-A15I,hla_a_03_01_19,hla_b_27_05_02,hla_c_01_02_01
15,TCGA-E2-A15I,hla_a_03_01_19,hla_b_27_05_02,hla_c_15_02_01
15,TCGA-E2-A15I,hla_a_26_01_03,hla_b_27_05_02,hla_c_01_02_01
15,TCGA-E2-A15I,hla_a_26_01_03,hla_b_27_05_02,hla_c_15_02_01
33,TCGA-B6-A1KN,hla_a_03_01_01_01,hla_b_27_05_02,hla_c_02_02_02
...,...,...,...,...
1064,TCGA-AR-A1AQ,hla_a_02_06_05,hla_b_27_02_01,hla_c_16_01_01
1096,TCGA-OL-A66H,hla_a_31_01_02,hla_b_27_05_02,hla_c_03_04_01_01
1096,TCGA-OL-A66H,hla_a_31_01_02,hla_b_27_05_02,hla_c_06_02_01_01
1096,TCGA-OL-A66H,hla_a_80_01_01_01,hla_b_27_05_02,hla_c_03_04_01_01


In [11]:
sub_ids = b27_sub['subject-id'].unique().tolist()
assays_for_sub = pqd.assays_for_patients(sub_ids, db_name='tcga-brca')
assays_for_sub

,subject-id,sample-id,assay-tech,assay-name,measurement-set-name
0,TCGA-A2-A0ER,tcga-a2-a0er-10a,:assay.technology/RNA-seq,RNA-seq data,immune-cell-deconvolution
1,TCGA-A2-A0ST,tcga-a2-a0st-10a,:assay.technology/RNA-seq,RNA-seq data,immune-cell-deconvolution
2,TCGA-A7-A13H,tcga-a7-a13h-01a,:assay.technology/SNP-array,cnv and variants,snp CNV data
3,TCGA-AN-A0AL,tcga-an-a0al-01a,:assay.technology/SNP-array,cnv and variants,snp CNV data
4,TCGA-A8-A099,tcga-a8-a099-01a,:assay.technology/WES,WES,baseline mutations
...,...,...,...,...,...
596,TCGA-A2-A04V,tcga-a2-a04v-01a,:assay.technology/RNA-seq,RNA-seq data,gx
597,TCGA-AC-A3W6,tcga-ac-a3w6-01a,:assay.technology/WES,WES,baseline mutations
598,TCGA-OL-A66H,tcga-ol-a66h-01a,:assay.technology/miscellaneous,Tumor purity,tumor purity
599,TCGA-E9-A1R6,tcga-e9-a1r6-01a,:assay.technology/RNA-seq,RNA-seq data,immune-cell-deconvolution


In [12]:
samples = pqd.samples("tcga-brca", db_name='tcga-brca')
samples

,sample-id,sample-tumor-type,sample-specimen,sample-metastasis,sample-uid,db-id,sample-timepoint-timepoint-id,sample-subject-subject-id,sample-study-day-study-day-id,sample-type-db-id,sample-gdc-anatomic-site-gdc-anatomic-site-name
0,tcga-a2-a0eu-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 1083184581b3c24ac85291d75a832b9a]",17592187518720,sample procurement,TCGA-A2-A0EU,35,17592186045420,Breast
1,tcga-a2-a0eu-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, e96e9d5e9e41bba015ba8e4daefda967]",17592187518803,sample procurement,TCGA-A2-A0EU,35,17592186045421,Breast
2,tcga-a7-a0d9-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 8c1dbb6ba31a61b3b978a1f93dccb86c]",17592187518823,sample procurement,TCGA-A7-A0D9,10,17592186045420,Breast
3,tcga-a7-a0d9-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, ba8add19b462211346377f6c7835cb6a]",17592187519258,sample procurement,TCGA-A7-A0D9,10,17592186045421,Breast
4,tcga-ao-a0jf-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 0bffbcada51f70f1978d207e22089d60]",17592187519626,sample procurement,TCGA-AO-A0JF,57,17592186045420,Breast
...,...,...,...,...,...,...,...,...,...,...,...
2291,tcga-b6-a402-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 9ede5e348369b0d13d4c7989eb565e3c]",17592188110342,sample procurement,TCGA-B6-A402,0,17592186045421,Breast
2292,tcga-e9-a1ne-11a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 85358f998e91a477db9bc868554ea46c]",17592188110466,sample procurement,TCGA-E9-A1NE,0,17592186045421,Breast
2293,tcga-ol-a5da-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 7882611517385fe91546199a73b523c7]",17592188111439,sample procurement,TCGA-OL-A5DA,33,17592186045421,Breast
2294,tcga-bh-a5j0-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 934e529d67c8a4b6e7840615131ce315]",17592188112629,sample procurement,TCGA-BH-A5J0,104,17592186045421,Breast


In [14]:
samples_sub = samples[samples["sample-subject-subject-id"].apply(lambda x: x in sub_ids)]
samples_sub

,sample-id,sample-tumor-type,sample-specimen,sample-metastasis,sample-uid,db-id,sample-timepoint-timepoint-id,sample-subject-subject-id,sample-study-day-study-day-id,sample-type-db-id,sample-gdc-anatomic-site-gdc-anatomic-site-name
69,tcga-e2-a15i-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 6418023f5ea1c0660b0ae8e57363a1d8]",17592187539851,sample procurement,TCGA-E2-A15I,0,17592186045420,Breast
70,tcga-e2-a15i-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 3e3a7b86768f0bff5fabc547366d9091]",17592187540016,sample procurement,TCGA-E2-A15I,0,17592186045421,Breast
71,tcga-e2-a15i-11a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 9538fcf5d7e50fa6e8304367b61d704a]",17592187540116,sample procurement,TCGA-E2-A15I,0,17592186045421,Breast
105,tcga-b6-a1kn-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, 83fe2368405c6e3d2a08288d83f8f0db]",17592187547478,sample procurement,TCGA-B6-A1KN,36,17592186045420,Breast
106,tcga-b6-a1kn-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 870c4800388cc94b29d58203db3a065b]",17592187548097,sample procurement,TCGA-B6-A1KN,36,17592186045421,Breast
...,...,...,...,...,...,...,...,...,...,...,...
2135,tcga-ar-a1aq-01a,breast,:sample.specimen/fresh-frozen,False,"[tcga-brca, ba15ce01b2d7006adcf93c4c164fd668]",17592188092973,sample procurement,TCGA-AR-A1AQ,8,17592186045420,Breast
2136,tcga-ar-a1aq-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 55cfb9a0147ff218289d4a58f76624a9]",17592188093360,sample procurement,TCGA-AR-A1AQ,8,17592186045421,Breast
2256,tcga-d8-a27e-10a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 3c207e60055076c99b42219f1cc01442]",17592188109129,sample procurement,TCGA-D8-A27E,41,17592186045421,Breast
2284,tcga-e9-a1n9-11a,NaN,:sample.specimen/fresh-frozen,False,"[tcga-brca, 93a83f1dd48b8968e49cd72d2eedac35]",17592188110096,sample procurement,TCGA-E9-A1N9,0,17592186045421,Breast


In [15]:
clin_sum = pqd.clinical_summary("tcga-brca", db_name='tcga-brca')
clin_sum

,clinical-observation-set-name
0,pfs os recurrence


In [29]:
samples_for_meas = assays_for_sub[assays_for_sub['measurement-set-name'] == 'baseline mutations']
samples_for_meas

,subject-id,sample-id,assay-tech,assay-name,measurement-set-name
4,TCGA-A8-A099,tcga-a8-a099-01a,:assay.technology/WES,WES,baseline mutations
28,TCGA-A7-A0DA,tcga-a7-a0da-01a,:assay.technology/WES,WES,baseline mutations
51,TCGA-AO-A12G,tcga-ao-a12g-01a,:assay.technology/WES,WES,baseline mutations
62,TCGA-A2-A0EO,tcga-a2-a0eo-01a,:assay.technology/WES,WES,baseline mutations
71,TCGA-A2-A04X,tcga-a2-a04x-01a,:assay.technology/WES,WES,baseline mutations
...,...,...,...,...,...
586,TCGA-E9-A1R4,tcga-e9-a1r4-01a,:assay.technology/WES,WES,baseline mutations
588,TCGA-A8-A07B,tcga-a8-a07b-01a,:assay.technology/WES,WES,baseline mutations
591,TCGA-A8-A06T,tcga-a8-a06t-01a,:assay.technology/WES,WES,baseline mutations
597,TCGA-AC-A3W6,tcga-ac-a3w6-01a,:assay.technology/WES,WES,baseline mutations


In [32]:
meas_samples = pqd.sample_measurements("tcga-brca", "baseline mutations",
                                       samples_for_meas['sample-id'].tolist(), db_name='tcga-brca',
                                       timeout=120)
meas_samples

,db-id,measurement-id,measurement-uid,measurement-sample-sample-id,measurement-variant-variant-id
0,17592234968693,tcga-a1-a0sh-01a-GRCh37:chr23:+:108911440:1089...,"[tcga-brca, a98f5fd31e7eb25a11aabcc29df6ebd7]",tcga-a1-a0sh-01a,GRCh37:chr23:+:108911440:108911440/C/T
1,17592234968695,tcga-a1-a0sh-01a-GRCh37:chr1:+:247040513:24704...,"[tcga-brca, 04a0aebe4f1e1662f62cee204cbc74b8]",tcga-a1-a0sh-01a,GRCh37:chr1:+:247040513:247040513/C/A
2,17592234968697,tcga-a1-a0sh-01a-GRCh37:chr6:+:85400295:854002...,"[tcga-brca, 2743c86cf741a47ec9aa882d755f01d8]",tcga-a1-a0sh-01a,GRCh37:chr6:+:85400295:85400295/G/A
3,17592234968699,tcga-a1-a0sh-01a-GRCh37:chr2:+:61965629:619656...,"[tcga-brca, edbd0160cf703fb987f0810ebcb98d62]",tcga-a1-a0sh-01a,GRCh37:chr2:+:61965629:61965629/T/C
4,17592234968701,tcga-a1-a0sh-01a-GRCh37:chr19:+:117865025:1178...,"[tcga-brca, cf54bb287c1d40d26e2b4af7551038de]",tcga-a1-a0sh-01a,GRCh37:chr19:+:117865025:117865025/C/G
...,...,...,...,...,...
8028,17592235128883,tcga-ol-a66h-01a-GRCh37:chr3:+:12447485:124474...,"[tcga-brca, 62f30b4472c51f6dedf626fa89c3cf34]",tcga-ol-a66h-01a,GRCh37:chr3:+:12447485:12447485/C/G
8029,17592235128885,tcga-ol-a66h-01a-GRCh37:chr8:+:10464643:104646...,"[tcga-brca, d584100cdf5d68946a6a47e3254d550e]",tcga-ol-a66h-01a,GRCh37:chr8:+:10464643:10464643/T/A
8030,17592235128887,tcga-ol-a66h-01a-GRCh37:chr17:+:19584880:19584...,"[tcga-brca, 23aa10c05ed127c8ea6ce17a3b3a6f29]",tcga-ol-a66h-01a,GRCh37:chr17:+:19584880:19584880/A/G
8031,17592235128889,tcga-ol-a66h-01a-GRCh37:chr4:+:88411538:884115...,"[tcga-brca, 985874c25fcacc499fb1b59e6eecf464]",tcga-ol-a66h-01a,GRCh37:chr4:+:88411538:88411538/C/G


In [34]:
var = pqr.variants(meas_samples['measurement-variant-variant-id'].tolist(), db_name='tcga-brca')
var

,variant-type,variant-ref-allele,variant-id,variant-alt-allele,db-id,variant-classification,variant-gene-gene-hgnc-symbol,variant-genomic-coordinates-genomic-coordinate-id,unify-import-most-recent-db-id
0,:variant.type/snp,C,GRCh37:chr23:+:108911440:108911440/C/T,T,17592187348366,:variant.classification/missense,CNTN4,GRCh37:chr23:+:108911440:108911440,17592186177812
1,:variant.type/snp,C,GRCh37:chr1:+:247040513:247040513/C/A,A,17592187348368,:variant.classification/nonsense,COL14A1,GRCh37:chr1:+:247040513:247040513,17592186177812
2,:variant.type/snp,G,GRCh37:chr6:+:85400295:85400295/G/A,A,17592187348370,:variant.classification/missense,COL7A1,GRCh37:chr6:+:85400295:85400295,17592186177812
3,:variant.type/snp,T,GRCh37:chr2:+:61965629:61965629/T/C,C,17592187348372,:variant.classification/missense,CPEB2,GRCh37:chr2:+:61965629:61965629,17592186177812
4,:variant.type/snp,C,GRCh37:chr19:+:117865025:117865025/C/G,G,17592187348374,:variant.classification/missense,CSMD1,GRCh37:chr19:+:117865025:117865025,17592186177812
...,...,...,...,...,...,...,...,...,...
8014,:variant.type/snp,C,GRCh37:chr3:+:12447485:12447485/C/G,G,17592187508556,:variant.classification/missense,PARP8,GRCh37:chr3:+:12447485:12447485,17592186177812
8015,:variant.type/snp,T,GRCh37:chr8:+:10464643:10464643/T/A,A,17592187508558,:variant.classification/missense,ASB9,GRCh37:chr8:+:10464643:10464643,17592186177812
8016,:variant.type/snp,A,GRCh37:chr17:+:19584880:19584880/A/G,G,17592187508560,:variant.classification/missense,ZNF660,GRCh37:chr17:+:19584880:19584880,17592186177812
8017,:variant.type/snp,C,GRCh37:chr4:+:88411538:88411538/C/G,G,17592187508562,:variant.classification/missense,SIPA1L2,GRCh37:chr4:+:88411538:88411538,17592186177812


In [36]:
w_var = meas_samples.merge(
    var[['variant-id',
         'variant-ref-allele',
         'variant-alt-allele',
         'variant-gene-gene-hgnc-symbol',
         'variant-genomic-coordinates-genomic-coordinate-id',
         'variant-type',
         'variant-classification']],
    left_on='measurement-variant-variant-id',
    right_on='variant-id',
    how='left'
)
w_var

,db-id,measurement-id,measurement-uid,measurement-sample-sample-id,measurement-variant-variant-id,variant-id,variant-ref-allele,variant-alt-allele,variant-gene-gene-hgnc-symbol,variant-genomic-coordinates-genomic-coordinate-id,variant-type,variant-classification
0,17592234968693,tcga-a1-a0sh-01a-GRCh37:chr23:+:108911440:1089...,"[tcga-brca, a98f5fd31e7eb25a11aabcc29df6ebd7]",tcga-a1-a0sh-01a,GRCh37:chr23:+:108911440:108911440/C/T,GRCh37:chr23:+:108911440:108911440/C/T,C,T,CNTN4,GRCh37:chr23:+:108911440:108911440,:variant.type/snp,:variant.classification/missense
1,17592234968695,tcga-a1-a0sh-01a-GRCh37:chr1:+:247040513:24704...,"[tcga-brca, 04a0aebe4f1e1662f62cee204cbc74b8]",tcga-a1-a0sh-01a,GRCh37:chr1:+:247040513:247040513/C/A,GRCh37:chr1:+:247040513:247040513/C/A,C,A,COL14A1,GRCh37:chr1:+:247040513:247040513,:variant.type/snp,:variant.classification/nonsense
2,17592234968697,tcga-a1-a0sh-01a-GRCh37:chr6:+:85400295:854002...,"[tcga-brca, 2743c86cf741a47ec9aa882d755f01d8]",tcga-a1-a0sh-01a,GRCh37:chr6:+:85400295:85400295/G/A,GRCh37:chr6:+:85400295:85400295/G/A,G,A,COL7A1,GRCh37:chr6:+:85400295:85400295,:variant.type/snp,:variant.classification/missense
3,17592234968699,tcga-a1-a0sh-01a-GRCh37:chr2:+:61965629:619656...,"[tcga-brca, edbd0160cf703fb987f0810ebcb98d62]",tcga-a1-a0sh-01a,GRCh37:chr2:+:61965629:61965629/T/C,GRCh37:chr2:+:61965629:61965629/T/C,T,C,CPEB2,GRCh37:chr2:+:61965629:61965629,:variant.type/snp,:variant.classification/missense
4,17592234968701,tcga-a1-a0sh-01a-GRCh37:chr19:+:117865025:1178...,"[tcga-brca, cf54bb287c1d40d26e2b4af7551038de]",tcga-a1-a0sh-01a,GRCh37:chr19:+:117865025:117865025/C/G,GRCh37:chr19:+:117865025:117865025/C/G,C,G,CSMD1,GRCh37:chr19:+:117865025:117865025,:variant.type/snp,:variant.classification/missense
...,...,...,...,...,...,...,...,...,...,...,...,...
8028,17592235128883,tcga-ol-a66h-01a-GRCh37:chr3:+:12447485:124474...,"[tcga-brca, 62f30b4472c51f6dedf626fa89c3cf34]",tcga-ol-a66h-01a,GRCh37:chr3:+:12447485:12447485/C/G,GRCh37:chr3:+:12447485:12447485/C/G,C,G,PARP8,GRCh37:chr3:+:12447485:12447485,:variant.type/snp,:variant.classification/missense
8029,17592235128885,tcga-ol-a66h-01a-GRCh37:chr8:+:10464643:104646...,"[tcga-brca, d584100cdf5d68946a6a47e3254d550e]",tcga-ol-a66h-01a,GRCh37:chr8:+:10464643:10464643/T/A,GRCh37:chr8:+:10464643:10464643/T/A,T,A,ASB9,GRCh37:chr8:+:10464643:10464643,:variant.type/snp,:variant.classification/missense
8030,17592235128887,tcga-ol-a66h-01a-GRCh37:chr17:+:19584880:19584...,"[tcga-brca, 23aa10c05ed127c8ea6ce17a3b3a6f29]",tcga-ol-a66h-01a,GRCh37:chr17:+:19584880:19584880/A/G,GRCh37:chr17:+:19584880:19584880/A/G,A,G,ZNF660,GRCh37:chr17:+:19584880:19584880,:variant.type/snp,:variant.classification/missense
8031,17592235128889,tcga-ol-a66h-01a-GRCh37:chr4:+:88411538:884115...,"[tcga-brca, 985874c25fcacc499fb1b59e6eecf464]",tcga-ol-a66h-01a,GRCh37:chr4:+:88411538:88411538/C/G,GRCh37:chr4:+:88411538:88411538/C/G,C,G,SIPA1L2,GRCh37:chr4:+:88411538:88411538,:variant.type/snp,:variant.classification/missense


In [37]:
w_var[w_var['variant-classification'] == ':variant.classification/missense']

,db-id,measurement-id,measurement-uid,measurement-sample-sample-id,measurement-variant-variant-id,variant-id,variant-ref-allele,variant-alt-allele,variant-gene-gene-hgnc-symbol,variant-genomic-coordinates-genomic-coordinate-id,variant-type,variant-classification
0,17592234968693,tcga-a1-a0sh-01a-GRCh37:chr23:+:108911440:1089...,"[tcga-brca, a98f5fd31e7eb25a11aabcc29df6ebd7]",tcga-a1-a0sh-01a,GRCh37:chr23:+:108911440:108911440/C/T,GRCh37:chr23:+:108911440:108911440/C/T,C,T,CNTN4,GRCh37:chr23:+:108911440:108911440,:variant.type/snp,:variant.classification/missense
2,17592234968697,tcga-a1-a0sh-01a-GRCh37:chr6:+:85400295:854002...,"[tcga-brca, 2743c86cf741a47ec9aa882d755f01d8]",tcga-a1-a0sh-01a,GRCh37:chr6:+:85400295:85400295/G/A,GRCh37:chr6:+:85400295:85400295/G/A,G,A,COL7A1,GRCh37:chr6:+:85400295:85400295,:variant.type/snp,:variant.classification/missense
3,17592234968699,tcga-a1-a0sh-01a-GRCh37:chr2:+:61965629:619656...,"[tcga-brca, edbd0160cf703fb987f0810ebcb98d62]",tcga-a1-a0sh-01a,GRCh37:chr2:+:61965629:61965629/T/C,GRCh37:chr2:+:61965629:61965629/T/C,T,C,CPEB2,GRCh37:chr2:+:61965629:61965629,:variant.type/snp,:variant.classification/missense
4,17592234968701,tcga-a1-a0sh-01a-GRCh37:chr19:+:117865025:1178...,"[tcga-brca, cf54bb287c1d40d26e2b4af7551038de]",tcga-a1-a0sh-01a,GRCh37:chr19:+:117865025:117865025/C/G,GRCh37:chr19:+:117865025:117865025/C/G,C,G,CSMD1,GRCh37:chr19:+:117865025:117865025,:variant.type/snp,:variant.classification/missense
5,17592234968703,tcga-a1-a0sh-01a-GRCh37:chr7:+:28506628:285066...,"[tcga-brca, 5d45e3e29c3ffaf60e4314c87ecb5241]",tcga-a1-a0sh-01a,GRCh37:chr7:+:28506628:28506628/G/T,GRCh37:chr7:+:28506628:28506628/G/T,G,T,CUBN,GRCh37:chr7:+:28506628:28506628,:variant.type/snp,:variant.classification/missense
...,...,...,...,...,...,...,...,...,...,...,...,...
8028,17592235128883,tcga-ol-a66h-01a-GRCh37:chr3:+:12447485:124474...,"[tcga-brca, 62f30b4472c51f6dedf626fa89c3cf34]",tcga-ol-a66h-01a,GRCh37:chr3:+:12447485:12447485/C/G,GRCh37:chr3:+:12447485:12447485/C/G,C,G,PARP8,GRCh37:chr3:+:12447485:12447485,:variant.type/snp,:variant.classification/missense
8029,17592235128885,tcga-ol-a66h-01a-GRCh37:chr8:+:10464643:104646...,"[tcga-brca, d584100cdf5d68946a6a47e3254d550e]",tcga-ol-a66h-01a,GRCh37:chr8:+:10464643:10464643/T/A,GRCh37:chr8:+:10464643:10464643/T/A,T,A,ASB9,GRCh37:chr8:+:10464643:10464643,:variant.type/snp,:variant.classification/missense
8030,17592235128887,tcga-ol-a66h-01a-GRCh37:chr17:+:19584880:19584...,"[tcga-brca, 23aa10c05ed127c8ea6ce17a3b3a6f29]",tcga-ol-a66h-01a,GRCh37:chr17:+:19584880:19584880/A/G,GRCh37:chr17:+:19584880:19584880/A/G,A,G,ZNF660,GRCh37:chr17:+:19584880:19584880,:variant.type/snp,:variant.classification/missense
8031,17592235128889,tcga-ol-a66h-01a-GRCh37:chr4:+:88411538:884115...,"[tcga-brca, 985874c25fcacc499fb1b59e6eecf464]",tcga-ol-a66h-01a,GRCh37:chr4:+:88411538:88411538/C/G,GRCh37:chr4:+:88411538:88411538/C/G,C,G,SIPA1L2,GRCh37:chr4:+:88411538:88411538,:variant.type/snp,:variant.classification/missense
